# CGRA 3rd Assignment - AR

Diogo Seabra Mota Henriques de Gouveia - 2018279473

Manuel Alberto Dionísio dos Santos - 2019231352

In [ ]:
%%html
<!-- ============================================ Background Color ============================================ -->
<head id="Background_Color">
    <style>
      body {
        background-color: rgb(46, 46, 46); /* You can change 'lightblue' to any color you want */
      }
    </style>
</head>

<!-- ============================================ Base Scripts ============================================ -->
<div id="Base Scripts">
    <script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecshader.js"></script>
    <script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecapp.js"></script>
    <script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/cgraobject.js"></script>
    <script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/cgratexture.js"></script>
    <script src='https://git.io/glm-js.min.js'></script>
    <script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/js-aruco/cv.js"></script>
    <script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/js-aruco/aruco.js"></script>
    <script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/js-aruco/svd.js"></script>
    <script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/js-aruco/posit1.js"></script>
</div>

<!-- ============================================ Get Video Input ============================================ -->
<video id="video" autoplay style="display: none; width:320px; height:240px;"></video>

<!-- ============================================ Shaders ============================================ -->
<div id="Shaders">

    <script id="vertex-shader" type="x-shader/x-vertex">
        precision mediump float;

        // === Inputs
        attribute  vec3 in_Position;
        attribute  vec3 in_Color;
        attribute  vec3 in_Normals;
        attribute vec2 in_Textures;

        // === Outputs to fragment shader
        varying  vec3 Colors;
        varying  vec3 Normals;
        varying  vec4 Position;
        varying  vec2 Textures;

        // === Uniforms
        uniform mat4 MVP;
        uniform mat3 NormalMatrix;
        uniform mat4 MV;

        void main()
        {
            Colors = in_Color;
            Normals = normalize(NormalMatrix * in_Normals);
            Position = MV * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);
            Textures = in_Textures;
            
            gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);
        }
    </script>

    <script id="fragment-shader" type="x-shader/x-fragment">
        precision mediump float;
    
        // === Inputs from vertex shader
        varying  vec3 Colors;         // -> Color of the vertex
        varying  vec3 Normals;        // -> Normal of the vertex
        varying  vec4 Position;      // -> Position of the vertex
        varying  vec2 Textures;

        // === Uniforms
        // - Textures
        uniform sampler2D uSampler;
        // - Lights
        uniform vec3 Ambient;           // -> Ambient light color
        uniform vec3 LightColor;        // -> Directional light color
        uniform vec3 LightPosition;    // -> Directional light direction
        uniform vec3 ViewPos;
        // - Materials
        uniform float Shininess;        // -> Shininess of the material (n - sharpness exponent, from 1 to 100)
        uniform float Strength;         // -> Strength of the reflection (Ks - specular coefficient, from 0.1 to 0.7)
        // - Attenuations
        uniform float ConstantAttenuation;
        uniform float LinearAttenuation;
        uniform float QuadraticAttenuation;
        
        
        void main(void)
        {
            // -- Auxiliar vectors
            vec3 LightDirection = LightPosition - vec3(Position);
            float LightDistance = length(LightDirection);
            LightDirection = normalize(LightDirection);
    
            // Model how much light is available for the fragment
            float attenuation = 1.0 / (ConstantAttenuation + LinearAttenuation * LightDistance + QuadraticAttenuation * LightDistance * LightDistance);
            
            vec3 ViewerDir = normalize(ViewPos - vec3(Position));
            vec3 HalfVector = normalize(LightDirection + ViewerDir);
            
            // Ambient part
            vec3 ambientLight = Ambient * Colors;

            // Diffused part
            float diffused_part = max(0.0, dot(Normals, LightDirection));
            vec3 scatteredLight = LightColor * (diffused_part * attenuation);
            
            // Specular part
            float specular_part = max(0.0, dot(Normals, HalfVector));
            if (diffused_part <= 0.0)
                specular_part = 0.0;
            else
                specular_part = pow(specular_part, Shininess)*Strength;
            vec3 reflectedLight = LightColor * (specular_part * attenuation);
    
            vec3 rgb = min((ambientLight + scatteredLight + reflectedLight), vec3(1.0));
            
            gl_FragColor = vec4(rgb, 1.0)*texture2D(uSampler, Textures);
        }
    </script>

    <script id="vertex-shader-video" type="x-shader/x-vertex">
        precision mediump float;

        attribute  vec3 in_Position;
        attribute  vec3 in_Color;
        attribute vec2 in_texcoords;
        uniform mat4 MVP;

        varying  vec3 ex_Color;
        varying  vec2 vTextureCoord;

        void main()
        {
            gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);
            vTextureCoord = in_texcoords;
            ex_Color = in_Color;
        }
    </script>

    <script id="fragment-shader-video" type="x-shader/x-fragment">
        precision mediump float;
        varying  vec2 vTextureCoord;
        varying  vec3 ex_Color;

        uniform sampler2D uSampler;

        void main()
        {
            gl_FragColor = texture2D(uSampler, vTextureCoord);
        }
    </script>

</div>

<!-- ============================================ Solids Definitions ============================================ -->
<div id="Solids Definitions">
    <!-- Cubes -->
    <script id="Cube">
        class cube extends CGRAobject
        {
            // ===== Constructor
            constructor(glcontext, color = [1.0, 1.0, 1.0])
            {
                // ===== Initialize the parent class
                super(glcontext); 
                
                // ===== Define Constants
                this.numvertices = 36, this.size = 1;
                let dim = this.size/2;
                this.vertices = [], this.colors = [], this.normals = [], this.texcoords = [];
                
                // ===== List vertices
                var verticesCube = [[ -dim, -dim,  dim],
                                    [-dim,  dim,  dim],
                                    [ dim,  dim,  dim],
                                    [ dim, -dim,  dim],
                                    [-dim, -dim, -dim],
                                    [-dim,  dim, -dim],
                                    [ dim,  dim, -dim],
                                    [ dim, -dim, -dim]];
                                    
                var normalsCube = [ [ 0,  0,  1],  // Front = +Z
                                    [ 1,  0,  0],  // Right = +X
                                    [ 0,  0, -1],  // Back  = -Z
                                    [-1,  0,  0],  // Left  = -X
                                    [ 0,  1,  0],  // Up    = +Y
                                    [ 0, -1,  0]]; // Down  = -Y

                var texturesCube = [[ 0.0, 0.0], //0
                                    [ 0.0, 1.0], //1
                                    [ 1.0, 1.0], //2
                                    [ 1.0, 0.0]];//3

                // ===== Lookup tables
                var indexes_vertex = [
                    0, 1, 2,  2, 3, 0,  // Front face
                    6, 7, 3,  2, 3, 6,  // Right face
                    6, 7, 4,  4, 5, 6,  // Back  face
                    1, 0, 4,  4, 5, 1,  // Left  face
                    1, 5, 6,  6, 2, 1,  // Up    face
                    0, 3, 7,  0, 4, 7]; // Down  face 

                var indexes_normals = [
                    0, 0, 0,  0, 0, 0,  // Front face
                    1, 1, 1,  1, 1, 1,  // Right face
                    2, 2, 2,  2, 2, 2,  // Back  face
                    3, 3, 3,  3, 3, 3,  // Left  face
                    4, 4, 4,  4, 4, 4,  // Up    face
                    5, 5, 5,  5, 5, 5]; // Down  face
                
                var indexes_textures = [
                    1, 0, 3,  3, 2, 1,  // Front face
                    3, 2, 1,  0, 1, 3,  // Right face
                    0, 1, 2,  2, 3, 0,  // Back  face
                    3, 2, 1,  1, 0, 3,  // Left  face
                    1, 0, 3,  3, 2, 1,  // Up    face
                    1, 2, 3,  1, 0, 3]; // Down  face
                
                // ===== Create structures to populate buffers
                for (var i = 0; i < this.numvertices; i++)
                {
                    this.vertices = this.vertices.concat(verticesCube[indexes_vertex[i]]);
                    this.normals = this.normals.concat(normalsCube[indexes_normals[i]]);
                    this.texcoords = this.texcoords.concat(texturesCube[indexes_textures[i]]);
                    this.colors.push(color);
                }
                
                // ===== Creating and configuring buffers
                this.vertexbuffer=this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.vertices), this.gl.STATIC_DRAW);
                
                this.colorbuffer = this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.colors.flat()), this.gl.STATIC_DRAW);

                this.normalsbuffer = this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.normalsbuffer);    
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.normals), this.gl.STATIC_DRAW);

                this.texcoordsbuffer=this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);  
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.texcoords), this.gl.STATIC_DRAW); 
            }
            
            // ===== Set texture
            setTexture(source)
            {
                this.texture = new CGRAtexture(this.gl);
                this.texture.load(source);
                this.textureid = this.texture.textureid;
            }
            
            // ===== Set material properties
            setMaterials(shiny = 50, strength = 0.4)
            {    
                this.Shininess = shiny;
                this.Strength = strength;
            }

            // ===== Function to dinamically change cube color
            setColor(color)
            {
                for (var i = 0; i < this.numvertices; i++)
                {
                    this.colors.push(color);
                }

                this.colorbuffer = this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.colors.flat()), this.gl.STATIC_DRAW);
            }
            
            // ===== Function to draw the cube
            drawit(viewMat, projectionMat,parentMat4 = glm.mat4(1.0))
            {
                this.shaderprog.startUsing();
                
                // ===== Setup textures
                this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,"in_Textures");
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
                this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                                2, // number of elements per attribute
                                this.gl.FLOAT,  // Type of elements
                                false,  // 
                                0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                                0); // Offset from the begining of a single vertex to this attribute
                this.gl.enableVertexAttribArray(this.texcoordsLocation);
                this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
                this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
                this.gl.uniform1i(this.textureLocation, 0);
                
                // ===== Normals into vertex shader
                this.normloc = this.gl.getAttribLocation(this.shaderprog.shaderProgram,"in_Normals");
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.normalsbuffer);
                this.gl.vertexAttribPointer(this.normloc, // Attribute location
                            3, // number of elements per attribute
                            this.gl.FLOAT,  // Type of elements
                            false,  // 
                            3*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                                0); // Offset from the begining of a single vertex to this attribute
                this.gl.enableVertexAttribArray(this.normloc);
                
                // ===== Normals transformation matrix
                var localT = parentMat4['*'](this.modelMat);
                this. NormMatloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"NormalMatrix");
                var NormMat = glm.mat3(glm.transpose(glm.inverse(localT)));
                this.gl.uniformMatrix3fv(this.NormMatloc, false, NormMat.array);

                // ===== Model transformation matrix
                this.Matloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"MV");
                var MV = viewMat['*'](parentMat4['*'](this.modelMat));
                this.gl.uniformMatrix4fv(this.Matloc, false, MV.array);
                
                // ===== Shininess value (Higher values result in smaller, sharper highlights, while lower values create larger, more diffused highlights. Usually in the range of 1 to 100.)
                this.Shininessloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Shininess");
                this.gl.uniform1f(this.Shininessloc, this.Shininess);

                // ===== Strength value
                this.Strengthloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Strength");
                this.gl.uniform1f(this.Strengthloc, this.Strength);

                
                this.shaderprog.stopUsing();

                // the parent method does the rest
                super.drawit(viewMat,projectionMat,parentMat4);
            }  
        }
    </script>

    <!-- Spheres -->
    <script id="Sphere">
        class sphere extends CGRAobject
        {
            // ===== Constructor
            constructor(glcontext, color = [1.0, 1.0, 1.0], faces = 20)
            {
                // ===== Initialize the parent class
                super(glcontext); 
                
                // ===== Define Constants
                // -- Number of faces (3 to inf)
                this.sides = faces;
                // -- Number of vertices
                this.numvertices = (6*this.sides + (6*this.sides)*(this.sides-2));
                this.length = 1; this.radius = 0.5;
                this.vertices = [], this.colors = [], this.normals = [], this.texCoords = [];
                
                // ===== Create circunferences (360/N sides)
                let slices = [];   
                for (let i = 0; i < this.sides; i++)
                { 
                    let theta = i*2*Math.PI/this.sides, aux = [];
                    for (let j = 0; j < this.sides-1; j++)
                    {     
                        aux.push(
                            [(Math.sin((j+1)*Math.PI/this.sides)*this.radius)*Math.cos(theta),
                            Math.cos((j+1)*Math.PI/this.sides)*this.radius,
                            (Math.sin((j+1)*Math.PI/this.sides)*this.radius)*Math.sin(theta)]);
                    }   
                    slices.push(aux);
                }
                slices.push([[0,this.radius,0],[0,-1*this.radius,0]]);
                
                // ===== Create vertex and color data to populate buffers
                for (let i = 0; i < this.sides; i++)
                {            
                    // Top side
                    this.vertices.push(slices[this.sides][0]);
                    this.vertices.push(slices[i][0]);
                    this.vertices.push(slices[(i+1)%this.sides][0]);
                    
                    this.normals.push(slices[this.sides][0]);
                    this.normals.push(slices[i][0]);
                    this.normals.push(slices[(i+1)%this.sides][0]);
                    
                    this.colors.push([color,color,color].flat());

                    this.texCoords.push([1-((i+(1/2))/(this.sides)), 0]);               // topo
                    this.texCoords.push([1-((i+0)/(this.sides)), 1/this.sides]);  // esquerda
                    this.texCoords.push([1-((i+1)/(this.sides)), 1/this.sides]);  // direita
                    
                    // Middle sections
                    for (let j = 0; j < this.sides - 2; j++)
                    {
                        // Triangle 1
                        this.vertices.push(slices[i][j]);
                        this.vertices.push(slices[(i+1)%this.sides][j]);
                        this.vertices.push(slices[i][j+1]);

                        this.normals.push(slices[i][j]);
                        this.normals.push(slices[(i+1)%this.sides][j]);
                        this.normals.push(slices[i][j+1]);
                        
                        this.colors.push([color,color,color].flat());

                        this.texCoords.push([1-((i+0)/(this.sides)), (j+1)/this.sides]); //a
                        this.texCoords.push([1-((i+1)/(this.sides)), (j+1)/this.sides]); //b
                        this.texCoords.push([1-((i+0)/(this.sides)), (j+2)/this.sides]); //c
                        
                        // Triangle 2
                        this.vertices.push(slices[i][j+1]);
                        this.vertices.push(slices[(i+1)%this.sides][j]);
                        this.vertices.push(slices[(i+1)%this.sides][j+1]);
                        
                        this.normals.push(slices[i][j+1]);
                        this.normals.push(slices[(i+1)%this.sides][j]);
                        this.normals.push(slices[(i+1)%this.sides][j+1]);
                        
                        this.colors.push([color,color,color].flat());

                        this.texCoords.push([1-((i+0)/(this.sides)), (j+2)/this.sides]); //c
                        this.texCoords.push([1-((i+1)/(this.sides)), (j+1)/this.sides]); //b
                        this.texCoords.push([1-((i+1)/(this.sides)), (j+2)/this.sides]); //d
                    }
                    
                    // Down side
                    this.vertices.push(slices[i][this.sides - 2]);
                    this.vertices.push(slices[(i+1)%this.sides][this.sides - 2]);
                    this.vertices.push(slices[this.sides][1]);
                    
                    this.normals.push(slices[i][this.sides - 2]);
                    this.normals.push(slices[(i+1)%this.sides][this.sides - 2]);
                    this.normals.push(slices[this.sides][1]);
                    
                    this.colors.push([color,color,color].flat());

                    this.texCoords.push([1-((i+0)/(this.sides)), 1-1/this.sides]);  // esquerda
                    this.texCoords.push([1-((i+1)/(this.sides)), 1-1/this.sides]); // direita
                    this.texCoords.push([1-((i+(1/2))/(this.sides)), 1]);            // bottom
                }
            
                // ===== Creating and configuring buffers
                this.vertexbuffer=this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.vertices.flat()), this.gl.STATIC_DRAW);
                
                this.colorbuffer = this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.colors.flat()), this.gl.STATIC_DRAW);

                this.normalsbuffer = this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.normalsbuffer);    
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.normals.flat()), this.gl.STATIC_DRAW);

                this.texcoordsbuffer=this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);  
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.texCoords.flat()), this.gl.STATIC_DRAW);
            }

            // ===== Set texture
            setTexture(source)
            {
                this.texture = new CGRAtexture(this.gl);
                this.texture.load(source);
                this.textureid = this.texture.textureid;
            }

            // ===== Set material properties
            setMaterials(shiny = 50, strength = 0.4)
            {
                this.Shininess = shiny;
                this.Strength = strength;
            }

            // ===== Function to dinamically change sphere color
            setColor(color)
            {
                for (var i = 0; i < this.numvertices; i++)
                {
                    this.colors.push(color);
                }

                this.colorbuffer = this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.colors.flat()), this.gl.STATIC_DRAW);
            }

            // ===== Function to draw the sphere
            drawit(viewMat, projectionMat,parentMat4 = glm.mat4(1.0))
            {
                this.shaderprog.startUsing();
                
                // ===== Setup textures
                this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,"in_Textures");
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
                this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                                2, // number of elements per attribute
                                this.gl.FLOAT,  // Type of elements
                                false,  // 
                                0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                                0); // Offset from the begining of a single vertex to this attribute
                this.gl.enableVertexAttribArray(this.texcoordsLocation);
                this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
                this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
                this.gl.uniform1i(this.textureLocation, 0);
                
                // ===== Normals into vertex shader
                this.normloc = this.gl.getAttribLocation(this.shaderprog.shaderProgram,"in_Normals");
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.normalsbuffer);
                this.gl.vertexAttribPointer(this.normloc, // Attribute location
                            3, // number of elements per attribute
                            this.gl.FLOAT,  // Type of elements
                            false,  // 
                            3*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                                0); // Offset from the begining of a single vertex to this attribute
                this.gl.enableVertexAttribArray(this.normloc);
                
                // ===== Normals transformation matrix
                var localT = parentMat4['*'](this.modelMat);
                this. NormMatloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"NormalMatrix");
                var NormMat = glm.mat3(glm.transpose(glm.inverse(localT)));
                this.gl.uniformMatrix3fv(this.NormMatloc, false, NormMat.array);

                // ===== Model transformation matrix
                this.Matloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"MV");
                var MV = viewMat['*'](parentMat4['*'](this.modelMat));
                this.gl.uniformMatrix4fv(this.Matloc, false, MV.array);
                
                // ===== Shininess value (Higher values result in smaller, sharper highlights, while lower values create larger, more diffused highlights. Usually in the range of 1 to 100.)
                this.Shininessloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Shininess");
                this.gl.uniform1f(this.Shininessloc, this.Shininess);

                // ===== Strength value
                this.Strengthloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Strength");
                this.gl.uniform1f(this.Strengthloc, this.Strength);
                
                this.shaderprog.stopUsing();

                // the parent method does the rest
                super.drawit(viewMat,projectionMat,parentMat4);
            }
        }
    </script>

    <!-- Cylinders -->
    <script id="Cylinder">
        class cylinder extends CGRAobject
        {
            // ===== Constructor
            constructor(glcontext, color = [1.0, 1.0, 1.0], faces = 20)
            {
                // Initialize the parent class
                super(glcontext); 
                
                // ===== Constants
                // Number of faces (3 to inf)
                this.sides = faces;
                this.numvertices = this.sides*6 + this.sides*3*2;
                this.length = 1; this.radius = 0.5;
                
                // ===== Calculate top and bottom bases coordinates
                let up_side = [], down_side = [], circleT = [];
                for (let i = 0; i < this.sides; i++)
                { 
                    let theta = i*2*Math.PI/this.sides;
                    up_side.push([this.radius*Math.cos(theta),
                                (this.length)/2,
                                this.radius*Math.sin(theta)]);
                    
                    down_side.push([this.radius*Math.cos(theta),
                                    -1*(this.length)/2,
                                    this.radius*Math.sin(theta)]);

                    circleT.push([0.25*Math.cos(theta)+0.25,
                                0.25*Math.sin(theta)+0.25]);
                }
                
                // ===== Calculate vertices and normals
                this.vertices = [], this.normals = [], this.texCoords = [], this.colors = [];
                for (let i = 0; i < this.sides; i++)
                {   
                    // Lateral
                    this.vertices.push(up_side[i]);
                    this.normals.push(up_side[i]);
                    this.vertices.push(up_side[(i+1)%this.sides]);
                    this.normals.push(up_side[(i+1)%this.sides]);
                    this.vertices.push(down_side[i]);
                    this.normals.push(down_side[i]);
                    this.texCoords.push([0.5+(i)*0.5/this.sides, 0.5]);
                    this.texCoords.push([0.5+(i+1)*0.5/this.sides, 0.5]);
                    this.texCoords.push([0.5+(i)*0.5/this.sides, 0]);
                    
                    this.vertices.push(down_side[i]);
                    this.normals.push(down_side[i]);
                    this.vertices.push(down_side[(i+1)%this.sides]);
                    this.normals.push(down_side[(i+1)%this.sides]);
                    this.vertices.push(up_side[(i+1)%this.sides]);
                    this.normals.push(up_side[(i+1)%this.sides]);
                    this.texCoords.push([0.5+(i)*0.5/this.sides, 0]);
                    this.texCoords.push([0.5+(i+1)*0.5/this.sides, 0]);
                    this.texCoords.push([0.5+(i+1)*0.5/this.sides, 0.5]);
                    
                    // Top
                    this.vertices.push(up_side[i]);
                    this.normals.push([0,1,0]);
                    this.vertices.push(up_side[(i+1)%this.sides]);
                    this.normals.push([0,1,0]);
                    this.vertices.push([0,(this.length)/2,0]);
                    this.normals.push([0,1,0]);
                    this.texCoords.push(circleT[i]);
                    this.texCoords.push(circleT[(i+1)%this.sides]);
                    this.texCoords.push([0.25,0.25]);
                    
                    // Base
                    this.vertices.push(down_side[i]);
                    this.normals.push([0,-1,0]);
                    this.vertices.push(down_side[(i+1)%this.sides]);
                    this.normals.push([0,-1,0]);
                    this.vertices.push([0,-1*(this.length)/2,0]);
                    this.normals.push([0,-1,0]);
                    this.texCoords.push(circleT[i]);
                    this.texCoords.push(circleT[(i+1)%this.sides]);
                    this.texCoords.push([0.25,0.25]);
                }

                // ===== Assign colors
                for (let i=0; i < (this.sides*6)*2; i++)
                {
                    this.colors.push(color);
                }
                
                // ===== Populate vertex and color buffers
                this.vertexbuffer=this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.vertices.flat()), this.gl.STATIC_DRAW);
                
                this.colorbuffer = this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.colors.flat()), this.gl.STATIC_DRAW);

                this.normalsbuffer = this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.normalsbuffer);    
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.normals.flat()), this.gl.STATIC_DRAW);

                this.texcoordsbuffer=this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);  
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.texCoords.flat()), this.gl.STATIC_DRAW);
            }

            // ===== Set texture
            setTexture(source)
            {
                this.texture = new CGRAtexture(this.gl);
                this.texture.load(source);
                this.textureid = this.texture.textureid;
            }

            // ===== Set material properties
            setMaterials(shiny = 50, strength = 0.4)
            {
                this.Shininess = shiny;
                this.Strength = strength;
            }

            // ===== Function to dinamically change cylinder color
            setColor(color)
            {
                for (var i = 0; i < this.numvertices; i++)
                {
                    this.colors.push(color);
                }

                this.colorbuffer = this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.colors.flat()), this.gl.STATIC_DRAW);
            }

            // ===== Function to draw the cylinder
            drawit(viewMat, projectionMat,parentMat4 = glm.mat4(1.0))
            {
                this.shaderprog.startUsing();
                
                // ===== Setup textures
                this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,"in_Textures");
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
                this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                                2, // number of elements per attribute
                                this.gl.FLOAT,  // Type of elements
                                false,  // 
                                0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                                0); // Offset from the begining of a single vertex to this attribute
                this.gl.enableVertexAttribArray(this.texcoordsLocation);
                this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
                this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
                this.gl.uniform1i(this.textureLocation, 0);
                
                // ===== Normals into vertex shader
                this.normloc = this.gl.getAttribLocation(this.shaderprog.shaderProgram,"in_Normals");
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.normalsbuffer);
                this.gl.vertexAttribPointer(this.normloc, // Attribute location
                            3, // number of elements per attribute
                            this.gl.FLOAT,  // Type of elements
                            false,  // 
                            3*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                                0); // Offset from the begining of a single vertex to this attribute
                this.gl.enableVertexAttribArray(this.normloc);
                
                // ===== Normals transformation matrix
                var localT = parentMat4['*'](this.modelMat);
                this. NormMatloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"NormalMatrix");
                var NormMat = glm.mat3(glm.transpose(glm.inverse(localT)));
                this.gl.uniformMatrix3fv(this.NormMatloc, false, NormMat.array);

                // ===== Model transformation matrix
                this.Matloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"MV");
                var MV = viewMat['*'](parentMat4['*'](this.modelMat));
                this.gl.uniformMatrix4fv(this.Matloc, false, MV.array);
                
                // ===== Shininess value (Higher values result in smaller, sharper highlights, while lower values create larger, more diffused highlights. Usually in the range of 1 to 100.)
                this.Shininessloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Shininess");
                this.gl.uniform1f(this.Shininessloc, this.Shininess);

                // ===== Strength value
                this.Strengthloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Strength");
                this.gl.uniform1f(this.Strengthloc, this.Strength);
                
                this.shaderprog.stopUsing();

                // the parent method does the rest
                super.drawit(viewMat,projectionMat,parentMat4);
            }
        }
    </script>

    <!-- Cones -->
    <script id="Cone">
        class cone extends CGRAobject
        {
            // ===== Constructor
            constructor(glcontext, color = [1.0, 1.0, 1.0], faces = 50)
            {
                // Initialize the parent class
                super(glcontext);

                // ===== Constants
                // Number of faces (3 to inf)
                this.sides = faces;
                this.numvertices = this.sides*3*2;
                this.length = 1; this.radius = 0.5;
                
                // ===== Calculate base
                var base = [], baseT = [];
                for (let i = 0; i < this.sides; i++)
                { 
                    let theta = i*2*Math.PI/this.sides;
                    base.push(
                        [this.radius*Math.cos(theta),
                        -1*(this.length)/2,
                        this.radius*Math.sin(theta)]);
                    baseT.push(
                        [this.radius*Math.cos(theta)+0.5,
                        this.radius*Math.sin(theta)+0.5]);
                }
                
                // ===== Calculate vertices and normals
                this.vertices = [], this.normals = [], this.texCoords = [], this.colors = [];
                for (let i = 0; i < this.sides; i++)
                {   
                    this.vertices.push(base[i]);
                    this.normals.push([0,-1,0]);
                    this.vertices.push(base[(i+1)%this.sides]);
                    this.normals.push([0,-1,0]);
                    this.vertices.push([0,-1*(this.length)/2,0]);
                    this.normals.push([0,-1,0]);
                    this.texCoords.push(baseT[i]);
                    this.texCoords.push(baseT[(i+1)%this.sides]);
                    this.texCoords.push([0.5,0.5]);

                    this.vertices.push(base[i]);
                    this.vertices.push(base[(i+1)%this.sides]);
                    this.vertices.push([0,(this.length)/2,0]);
                    
                    var a = glm.vec3(base[(i+1)%this.sides][0] - base[i][0],
                                    base[(i+1)%this.sides][1] - base[i][1],
                                    base[(i+1)%this.sides][2] - base[i][2]);

                    var b = glm.vec3(0 - base[i][0],
                                    (this.length)/2 - base[i][1],
                                    0 - base[i][2]);

                    var c = glm.cross(b,a);

                    this.normals.push(c.array);
                    this.normals.push(c.array);
                    this.normals.push(c.array);
                    this.texCoords.push(baseT[i]);
                    this.texCoords.push(baseT[(i+1)%this.sides]);
                    this.texCoords.push([0.5,0.5]);
                }
                
                // ===== Assign colors
                for (let i=0; i < (this.sides*6); i++)
                {
                    this.colors.push(color);
                }
                
                // ===== Creating and configuring buffers
                this.vertexbuffer=this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.vertices.flat()), this.gl.STATIC_DRAW);
                
                this.colorbuffer = this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.colors.flat()), this.gl.STATIC_DRAW);

                this.normalsbuffer = this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.normalsbuffer);    
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.normals.flat()), this.gl.STATIC_DRAW);

                this.texcoordsbuffer=this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);  
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.texCoords.flat()), this.gl.STATIC_DRAW);
            }

            // ===== Set texture
            setTexture(source)
            {
                this.texture = new CGRAtexture(this.gl);
                this.texture.load(source);
                this.textureid = this.texture.textureid;
            }

            // ===== Set material properties
            setMaterials(shiny = 50, strength = 0.4)
            {
                this.Shininess = shiny;
                this.Strength = strength;
            }

            // ===== Function to dinamically change cone color
            setColor(color)
            {
                for (var i = 0; i < this.numvertices; i++)
                {
                    this.colors.push(color);
                }

                this.colorbuffer = this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.colors.flat()), this.gl.STATIC_DRAW);
            }

            // ===== Function to draw the cone
            drawit(viewMat, projectionMat,parentMat4 = glm.mat4(1.0))
            {
                this.shaderprog.startUsing();
                
                // ===== Setup textures
                this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,"in_Textures");
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
                this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                                2, // number of elements per attribute
                                this.gl.FLOAT,  // Type of elements
                                false,  // 
                                0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                                0); // Offset from the begining of a single vertex to this attribute
                this.gl.enableVertexAttribArray(this.texcoordsLocation);
                this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
                this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
                this.gl.uniform1i(this.textureLocation, 0);
                
                // ===== Normals into vertex shader
                this.normloc = this.gl.getAttribLocation(this.shaderprog.shaderProgram,"in_Normals");
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.normalsbuffer);
                this.gl.vertexAttribPointer(this.normloc, // Attribute location
                            3, // number of elements per attribute
                            this.gl.FLOAT,  // Type of elements
                            false,  // 
                            3*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                                0); // Offset from the begining of a single vertex to this attribute
                this.gl.enableVertexAttribArray(this.normloc);
                
                // ===== Normals transformation matrix
                var localT = parentMat4['*'](this.modelMat);
                this. NormMatloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"NormalMatrix");
                var NormMat = glm.mat3(glm.transpose(glm.inverse(localT)));
                this.gl.uniformMatrix3fv(this.NormMatloc, false, NormMat.array);

                // ===== Model transformation matrix
                this.Matloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"MV");
                var MV = viewMat['*'](parentMat4['*'](this.modelMat));
                this.gl.uniformMatrix4fv(this.Matloc, false, MV.array);
                
                // ===== Shininess value (Higher values result in smaller, sharper highlights, while lower values create larger, more diffused highlights. Usually in the range of 1 to 100.)
                this.Shininessloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Shininess");
                this.gl.uniform1f(this.Shininessloc, this.Shininess);

                // ===== Strength value
                this.Strengthloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Strength");
                this.gl.uniform1f(this.Strengthloc, this.Strength);
                
                this.shaderprog.stopUsing();

                // the parent method does the rest
                super.drawit(viewMat,projectionMat,parentMat4);
            }
        }
    </script>

    <!-- Discs -->
    <!-- <script id="Disc">
        class disc extends CGRAobject
        {
            // ===== Constructor
            constructor(glcontext, color = [1.0, 1.0, 1.0], faces = 20)
            {}

            // ===== Set texture
            setTexture(source)
            {
                this.texture = new CGRAtexture(this.gl);
                this.texture.load(source);
                this.textureid = this.texture.textureid;
            }

            // ===== Set material properties
            setMaterials(shiny = 50, strength = 0.4)
            {
                this.Shininess = shiny;
                this.Strength = strength;
            }

            // ===== Function to dinamically change disc color
            setColor(color)
            {
                for (var i = 0; i < this.numvertices; i++)
                {
                    this.colors.push(color);
                }

                this.colorbuffer = this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.colors.flat()), this.gl.STATIC_DRAW);
            }

            // ===== Function to draw the disc
            drawit(viewMat, projectionMat,parentMat4 = glm.mat4(1.0))
            {}
        }
    </script> -->

    <!-- Square -->
    <script id="Square">
        class square extends CGRAobject
        {
            constructor(glcontext)
            {
                super(glcontext);
                this.numvertices = 6;
                this.vertices = [
                    -1.0, -1.0, 0.0,
                     1.0, -1.0, 0.0,
                     1.0,  1.0, 0.0,
                     1.0,  1.0, 0.0,
                    -1.0,  1.0, 0.0,
                    -1.0, -1.0, 0.0
                ];
    
                this.colors = [
                    1.0, 1.0, 1.0, 1.0,  // white
                    1.0, 1.0, 1.0, 1.0,  // white
                    1.0, 1.0, 1.0, 1.0,  // white
                    1.0, 1.0, 1.0, 1.0,  // white
                    1.0, 1.0, 1.0, 1.0,  // white
                    1.0, 1.0, 1.0, 1.0   // white
                ];
    
                this.texCoords = [
                    0.0, 1.0,
                    1.0, 1.0,
                    1.0, 0.0,
                    1.0, 0.0,
                    0.0, 0.0,
                    0.0, 1.0
                ];
    
                this.vertexbuffer=this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.vertices), this.gl.STATIC_DRAW);4
    
                this.colorbuffer = this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.colors), this.gl.STATIC_DRAW);
    
                this.texcoordsbuffer=this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.texCoords), this.gl.STATIC_DRAW);
            }
    
            settexture(cgratex, mirror = 0){
                this.textureid = cgratex.textureid;
                
                // Mirrored texture (default)
                if(mirror == 0)
                {
                    this.texCoords = [
                        0.0, 1.0,
                        1.0, 1.0,
                        1.0, 0.0,
                        1.0, 0.0,
                        0.0, 0.0,
                        0.0, 1.0
                    ];
                }
                // "Natural" texture
                else
                {
                    this.texCoords = [
                        1.0, 1.0,
                        0.0, 1.0,
                        0.0, 0.0,
                        0.0, 0.0,
                        1.0, 0.0,
                        1.0, 1.0
                    ];
                }
                this.texcoordsbuffer=this.gl.createBuffer();
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);
                this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.texCoords), this.gl.STATIC_DRAW);
            }
    
            setShader(shader){
                this.shaderprog = shader;
            }
    
            drawit(viewMat, projectionMat){
                this.shaderprog.startUsing();
                
                this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,"in_texcoords");
                
                this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
                this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                                2, // number of elements per attribute
                                this.gl.FLOAT,  // Type of elements
                                false,  // 
                                0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                                0); // Offset from the begining of a single vertex to this attribute
                this.gl.enableVertexAttribArray(this.texcoordsLocation);
                this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
                this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
                this.gl.uniform1i(this.textureLocation, 0);
                // the parent method does the rest
                super.drawit(viewMat,projectionMat);
                
            }
        }
    </script>

</div>

<center id="See_Solids">
    <div id="See_Solids">
        <canvas id="Testes" width="600" height="600"
            style="border:2px solid #000000;">
            Error: Your browser does not support the HTML canvas tag.
        </canvas>
        <script id="Testes">
            class Testes_App extends DEECapp
            {
                counter = 0;

                initialize()
                {
                    // ===== Perform initializations
                    this.gl.enable(this.gl.DEPTH_TEST);
                    this.gl.clearColor(0.1, 0.1, 0.1, 1.0);
                    
                    // ===== Initialize shader
                    var fragsrc = document.getElementById("fragment-shader").text;
                    var vertsrc = document.getElementById("vertex-shader").text;
                    this.shaderprog = new DEECshader(this.gl);
                    this.shaderprog.srcShaders(vertsrc,fragsrc);
                    
                    // ===== Instantiate cubes
                    this.myCube = new cube(this.gl);
                    this.myCube.setShader(this.shaderprog);
                    this.myCube.setMaterials(100,10);
                    this.myCube.setTexture("Diamond.jpg");
                    
                    this.mySphere = new sphere(this.gl);
                    this.mySphere.setShader(this.shaderprog);
                    this.mySphere.setMaterials(10,10);
                    this.mySphere.setTexture("EarTex.jpg");

                    this.myCylinder = new cylinder(this.gl);
                    this.myCylinder.setShader(this.shaderprog);
                    this.myCylinder.setMaterials(10,10);
                    this.myCylinder.setTexture("CyTex.png");

                    this.myCone = new cone(this.gl);
                    this.myCone.setShader(this.shaderprog);
                    this.myCone.setMaterials(10,10);
                    this.myCone.setTexture("ConeTex.png");


                    // ===== Set camera
                    this.viewpos = glm.vec3(0, 0, 6);

                    // ===== Set lights
                    this.setLights();

                    ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
                    //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
                    //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
                    //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
                    //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
                    ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
                    this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,100);      
                    
                    //////////////////////////////////////////////////////////////////////////
                    //glm.vec3(3, 3, 5): A posição da câmera no espaço 3D.                  //
                    //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
                    //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
                    //////////////////////////////////////////////////////////////////////////
                    this.viewM = glm.lookAt(this.viewpos,glm.vec3(0,0,0),glm.vec3(0,1,0));        
                    
                    
                }

                setLights()
                {
                    // ===== Set parameters of light
                    this.shaderprog.startUsing();
                    // -- Ambient Light
                    this.Ambientloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Ambient");
                    var ambient_light = glm.vec3(0.2);
                    this.gl.uniform3fv(this.Ambientloc, ambient_light.array);
                    // -- Directional Light Color
                    this.LightColorloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightColor");
                    var color_light = glm.vec3(1.0);
                    this.gl.uniform3fv(this.LightColorloc, color_light.array);
                    // -- Directional Light Position
                    this.LightPosloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightPosition");
                    var light_pos = glm.vec3(2.0, 2.0, 2.0);
                    this.gl.uniform3fv(this.LightPosloc, light_pos.array);
                    // -- View Position
                    this.ViewPosloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ViewPos");
                    this.gl.uniform3fv(this.ViewPosloc, this.viewpos.array);
                    
                    // ----- Attenuations
                    // -- Constant Attenuation
                    this.ConstantAttenuationloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ConstantAttenuation");
                    this.gl.uniform1f(this.ConstantAttenuationloc, 1.0);
                    // -- Linear Attenuation
                    this.LinearAttenuationloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LinearAttenuation");
                    this.gl.uniform1f(this.LinearAttenuationloc, 0.09);
                    // -- Quadratic Attenuation
                    this.QuadraticAttenuationloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"QuadraticAttenuation");
                    this.gl.uniform1f(this.QuadraticAttenuationloc, 0.032);
                    // -- Setup done
                    this.shaderprog.stopUsing();
                }

                render()
                {
                    this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
                    this.counter++;
                
                    // ==================== Cube ====================
                    // ===== Model
                    var model = glm.mat4(glm.mat3(1.0));
                    // ===== Translations
                    var translation = glm.vec3(-1.0, 1.0, 0.0);
                    model = glm.translate(model, translation);
                    // ===== Rotation
                    var rotation = glm.radians(this.counter);
                    var axis = glm.vec3(0.0, 1.0, 0.0);
                    model = glm.rotate(model, rotation, axis);
                    rotation = glm.radians(this.counter*2);
                    axis = glm.vec3(0.0, 0.0, 1.0);
                    model = glm.rotate(model, rotation, axis);
                    // ===== Final Transformation
                    this.myCube.setModelTransformation(model); 
                    // ===== Draw Cube 
                    this.myCube.drawit(this.viewM,this.projectionM);

                    // ==================== Sphere ====================
                    // ===== Model
                    var model = glm.mat4(glm.mat3(1.0));
                    // ===== Translations
                    var translation = glm.vec3(1.0, 1.0, 0.0);
                    model = glm.translate(model, translation);
                    // ===== Rotation
                    var rotation = glm.radians(this.counter);
                    var axis = glm.vec3(0.0, 1.0, 0.0);
                    // model = glm.rotate(model, rotation, axis);
                    // rotation = glm.radians(180);
                    // axis = glm.vec3(0.0, 0.0, 1.0);
                    model = glm.rotate(model, rotation, axis);
                    // ===== Final Transformation
                    this.mySphere.setModelTransformation(model);
                    // ===== Draw Sphere
                    this.mySphere.drawit(this.viewM,this.projectionM);

                    // ==================== Cylinder ====================
                    // ===== Model
                    var model = glm.mat4(glm.mat3(1.0));
                    // ===== Translations
                    var translation = glm.vec3(-1.0, -1.0, 0.0);
                    model = glm.translate(model, translation);
                    // ===== Rotation
                    var rotation = glm.radians(this.counter);
                    var axis = glm.vec3(1.0, 0.0, 0.0);
                    model = glm.rotate(model, rotation, axis);
                    var rotation = glm.radians(this.counter*2);
                    var axis = glm.vec3(0.0, 0.0, 1.0);
                    model = glm.rotate(model, rotation, axis);
                    // ===== Final Transformation
                    this.myCylinder.setModelTransformation(model);
                    // ===== Draw Cylinder
                    this.myCylinder.drawit(this.viewM,this.projectionM);

                    // ==================== Cone ====================
                    // ===== Model
                    var model = glm.mat4(glm.mat3(1.0));
                    // ===== Translations
                    var translation = glm.vec3(1.0, -1.0, 0.0);
                    model = glm.translate(model, translation);
                    // ===== Rotation
                    var rotation = glm.radians(this.counter*3);
                    var axis = glm.vec3(1.0, 0.0, 0.0);
                    model = glm.rotate(model, rotation, axis);
                    var rotation = glm.radians(-this.counter*2);
                    var axis = glm.vec3(0.0, 0.0, 1.0);
                    model = glm.rotate(model, rotation, axis);
                    // ===== Final Transformation
                    this.myCone.setModelTransformation(model);
                    // ===== Draw Cone
                    this.myCone.drawit(this.viewM,this.projectionM);

                }
            }

            var app = new Testes_App('Testes');

            app.run();

        </script>
    </div>
</center>

<!-- ============================================ Solar System Definition ============================================ -->
<center id="SolarSystem">
    <div id="SolarSystem">
        <canvas id="SolarSystem_Canvas" width="600" height="600"
            style="border:2px solid #000000;">
            Error: Your browser does not support the HTML canvas tag.
        </canvas>

        <script id="SolarSystem">
            class solarSystem extends CGRAobject
                {
                    form = 1;
                    constructor(glcontext, form = 1, sunFlg = 1)
                    {    
                        super(glcontext)

                        switch(form)
                        {
                            case 1: 
                                this.Sun = new sphere(this.gl,[1, 0.7, 0] , 20);
                                this.Mercury = new sphere(this.gl,[0.58, 0.58, 0.58] , 20);
                                this.Venus = new sphere(this.gl,[0.58, 0.29, 0.1] , 20);
                                this.Earth = new sphere(this.gl,[0, 0.8, 0.3], 20);
                                this.Mars = new sphere(this.gl,[0.9, 0, 0], 20);
                                this.Jupiter = new sphere(this.gl,[0.9, 0.4, 0], 20);
                                this.Saturn = new sphere(this.gl,[0.8, 0.4, 0], 20);
                                this.Uranus = new sphere(this.gl,[0.4, 0.6, 0.9], 20);
                                this.Neptune = new sphere(this.gl,[0, 0, 1], 20);
                                break;
                            case 2:
                                this.Sun = new cube(this.gl);
                                this.Mercury = new cube(this.gl);
                                this.Venus = new cube(this.gl);
                                this.Earth = new cube(this.gl);
                                this.Mars = new cube(this.gl);
                                this.Jupiter = new cube(this.gl);
                                this.Saturn = new cube(this.gl);
                                this.Uranus = new cube(this.gl);
                                this.Neptune = new cube(this.gl);
                                break;
                            case 3:
                                this.Sun = new cylinder(this.gl,[1, 0.7, 0] , 10);
                                this.Mercury = new cylinder(this.gl,[0.58, 0.58, 0.58] , 10);
                                this.Venus = new cylinder(this.gl,[0.58, 0.29, 0.1], 10);
                                this.Earth = new cylinder(this.gl,[0, 0.8, 0.3], 10);
                                this.Mars = new cylinder(this.gl,[0.9, 0, 0] , 10);
                                this.Jupiter = new cylinder(this.gl,[0.9, 0.4, 0] , 10);
                                this.Saturn = new cylinder(this.gl,[0.8, 0.4, 0] , 10);
                                this.Uranus = new cylinder(this.gl,[0.4, 0.6, 0.9] , 10);
                                this.Neptune = new cylinder(this.gl,[0, 0, 1] , 10);
                                break;
                            case 4:
                                this.Sun = new cone(this.gl,[1, 0.7, 0]);
                                this.Mercury = new cone(this.gl,[0.58, 0.58, 0.58]);
                                this.Venus = new cone(this.gl,[0.58, 0.29, 0.1]);
                                this.Earth = new cone(this.gl,[0, 0.8, 0.3]);
                                this.Mars = new cone(this.gl,[0.9, 0, 0]);
                                this.Jupiter = new cone(this.gl,[0.9, 0.4, 0]);
                                this.Saturn = new cone(this.gl,[0.8, 0.4, 0]);
                                this.Uranus = new cone(this.gl,[0.4, 0.6, 0.9]);
                                this.Neptune = new cone(this.gl,[0, 0, 1]);
                                break;
                        }

                        var Color = [0.3, 0.3, 0.3];
                        switch(sunFlg)
                        {
                            case 1:
                                Color = [0.3, 0.3, 0.3];
                                break;
                            case 2:
                                Color = [0.9, 0.9, 0.9];
                                break;
                            case 3:
                                Color = [1, 0.7, 0.0];
                                break;
                            case 4:
                                Color = [1, 0.0, 0.0];
                                break;
                        }
                        // this.SaturnDisc = new disc(this.gl, [0.9, 0.7, 0] ,[0.7, 0.7, 0.6], 20);
                        
                        this.Sun.setTexture("SunTex.jpg");
                        this.Mercury.setTexture("MerTex.jpg");
                        this.Venus.setTexture("VenTex.jpg");
                        this.Earth.setTexture("EarTex.jpg");
                        this.Mars.setTexture("MarTex.jpg");
                        this.Jupiter.setTexture("JupTex.jpg");
                        this.Saturn.setTexture("SatTex.jpg");
                        this.Uranus.setTexture("UraTex.jpg");
                        this.Neptune.setTexture("NepTex.jpg");
                        // this.SaturnDisc.setTexture("SatTex.jpg");

                        this.Sun.setColor(Color)
                    }
                        
                    setShader(shader)
                    {
                        this.Sun.setShader(shader);
                        this.Mercury.setShader(shader);
                        this.Venus.setShader(shader);
                        this.Earth.setShader(shader);
                        this.Mars.setShader(shader);
                        this.Jupiter.setShader(shader);
                        this.Saturn.setShader(shader);
                        this.Uranus.setShader(shader);
                        this.Neptune.setShader(shader);
                        // this.SaturnDisc.setShader(shader);
                    }

                    setMaterials(shiny = 50, strength = 0.4)
                    {
                        this.Sun.setMaterials(shiny, strength);
                        this.Mercury.setMaterials(shiny, strength);
                        this.Venus.setMaterials(shiny, strength);
                        this.Earth.setMaterials(shiny, strength);
                        this.Mars.setMaterials(shiny, strength);
                        this.Jupiter.setMaterials(shiny, strength);
                        this.Saturn.setMaterials(shiny, strength);
                        this.Uranus.setMaterials(shiny, strength);
                        this.Neptune.setMaterials(shiny, strength);
                        // this.SaturnDisc.setMaterials(shiny, strength);
                    }

                    drawit(viewM, projectM, counter, parentMat4 = glm.mat4(1.0))
                    {
                        var globalMat = parentMat4['*'](this.modelMat);
                        
                        ////////////////////////SUN//////////////////////////////
                        var Sun = glm.mat4(glm.mat3(1.0));
                        // ===== Scale
                        var scale = glm.vec3(2.5,2.5, 2.5);
                        Sun = glm.scale(Sun, scale);
                        // ===== Rotation
                        var rotation = glm.radians(-counter);
                        var axis = glm.vec3(0.0,1.0,0.0);
                        Sun = glm.rotate(Sun, rotation, axis);
                        // ===== Translate
                        var translation = glm.vec3(0,0,0);
                        Sun = glm.translate(Sun, translation);
                        // ===== Final Transformation
                        this.Sun.setModelTransformation(Sun);      
                        // ===== Draw
                        this.Sun.drawit(viewM,projectM, globalMat);
                        
                        
                        //////////////////////MERCURY///////////////////////////////////
                        var Mercury = glm.mat4(glm.mat3(1));
                        // ===== Scale
                        var scale = glm.vec3(0.024, 0.024, 0.024);
                        Mercury = glm.scale(Mercury, scale);
                        // ===== Rotation
                        var outer_rotation = counter*365/85;
                        var rotation = glm.radians(outer_rotation);
                        var axis = glm.vec3(0.0,1.0,0.0);
                        Mercury = glm.rotate(Mercury, rotation, axis);
                        // ===== Translate
                        var translation = glm.vec3(0,0,-70);
                        Mercury = glm.translate(Mercury, translation);
                        // ===== Rotation
                        Mercury = glm.rotate(Mercury, rotation, axis);
                        // ===== Final Transformation
                        this.Mercury.setModelTransformation(Mercury);      
                        // ===== Draw
                        this.Mercury.drawit(viewM,projectM, globalMat);
                        
                        
                        /////////////////////VENUS/////////////////////////////////////
                        var Venus = glm.mat4(glm.mat3(1));
                        // ===== Scale
                        var scale = glm.vec3(0.121/2, 0.121/2, 0.121/2);
                        Venus = glm.scale(Venus, scale);
                        // ===== Rotation
                        var outer_rotation = counter*365/224;
                        var rotation = glm.radians(outer_rotation);
                        var axis = glm.vec3(0.0,1.0,0.0);
                        Venus = glm.rotate(Venus, rotation, axis);
                        // ===== Translate
                        var translation = glm.vec3(0,0,-40);
                        Venus = glm.translate(Venus, translation);
                        // ===== Rotation
                        var inner_rotation = counter/365;
                        var rotation = glm.radians(inner_rotation);
                        Venus = glm.rotate(Venus, rotation, axis);
                        // ===== Final Transformation
                        this.Venus.setModelTransformation(Venus);      
                        // ===== Draw
                        this.Venus.drawit(viewM,projectM, globalMat);
                        
                        
                        ///////////////////////EARTH/////////////////////////////////////
                        var Earth = glm.mat4(glm.mat3(1));
                        // ===== Scale
                        var scale = glm.vec3(0.127/2, 0.127/2, 0.127/2);
                        Earth = glm.scale(Earth, scale);
                        // ===== Rotation
                        var outer_rotation = counter*365/365;
                        var rotation = glm.radians(outer_rotation);
                        var axis = glm.vec3(0.0,1.0,0.0);
                        Earth = glm.rotate(Earth, rotation, axis);
                        // ===== Translate
                        var translation = glm.vec3(0,0,-50);
                        Earth = glm.translate(Earth, translation);
                        // ===== Rotation
                        var inner_rotation = counter/365;
                        var rotation = glm.radians(inner_rotation);
                        Earth = glm.rotate(Earth, rotation, axis);
                        // ===== Final Transformation
                        this.Earth.setModelTransformation(Earth);      
                        // ===== Draw
                        this.Earth.drawit(viewM,projectM, globalMat);
                        
                        
                        ///////////////////////MARS/////////////////////////////////
                        var Mars = glm.mat4(glm.mat3(1));
                        // ===== Scale
                        var scale = glm.vec3(0.067/2, 0.067/2, 0.067/2);
                        Mars = glm.scale(Mars, scale);
                        // ===== Rotation
                        var outer_rotation = counter*365/(365*1.88);
                        var rotation = glm.radians(outer_rotation);
                        var axis = glm.vec3(0.0,1.0,0.0);
                        Mars = glm.rotate(Mars, rotation, axis);
                        // ===== Translate
                        var translation = glm.vec3(0,0,-120);
                        Mars = glm.translate(Mars, translation);
                        // ===== Rotation
                        Mars = glm.rotate(Mars, rotation, axis);
                        // ===== Final Transformation
                        this.Mars.setModelTransformation(Mars);      
                        // ===== Draw
                        this.Mars.drawit(viewM,projectM, globalMat);
                        
                        
                        //////////////////////JUPITER///////////////////////////////
                        var Jupiter = glm.mat4(glm.mat3(1));
                        // ===== Scale
                        var scale = glm.vec3(1.42/2, 1.42/2, 1.42/2);
                        Jupiter = glm.scale(Jupiter, scale);
                        // ===== Rotation
                        var outer_rotation = counter/11.86;
                        var rotation = glm.radians(outer_rotation);
                        var axis = glm.vec3(0.0,1.0,0.0);
                        Jupiter = glm.rotate(Jupiter, rotation, axis);
                        // ===== Translate
                        var translation = glm.vec3(0,0,-8);
                        Jupiter = glm.translate(Jupiter, translation);
                        // ===== Rotation
                        Jupiter = glm.rotate(Jupiter, rotation, axis);
                        // ===== Final Transformation
                        this.Jupiter.setModelTransformation(Jupiter);      
                        // ===== Draw
                        this.Jupiter.drawit(viewM,projectM, globalMat);
                        
                        
                        //////////////////////SATURN///////////////////////////////

                        var Saturn = glm.mat4(glm.mat3(1));
                        // ===== Scale
                        var scale = glm.vec3(1.2/2, 1.2/2, 1.2/2);
                        Saturn = glm.scale(Saturn, scale);
                        // ===== Rotation
                        var outer_rotation = counter/29.46;
                        var rotation = glm.radians(outer_rotation);
                        var axis = glm.vec3(0.0,1.0,0.0);
                        Saturn = glm.rotate(Saturn, rotation, axis);
                        // ===== Translate
                        var translation = glm.vec3(0,0,-12);
                        Saturn = glm.translate(Saturn, translation);
                        // ===== Rotation
                        Saturn = glm.rotate(Saturn, rotation, axis);
                        // ===== Final Transformation
                        this.Saturn.setModelTransformation(Saturn);
                        var scale = glm.vec3(1.5, 1.5, 1.5);
                        Saturn = glm.scale(Saturn, scale);
                        // this.SaturnDisc.setModelTransformation(Saturn);
                        
                        // ===== Draw
                        this.Saturn.drawit(viewM,projectM, globalMat);
                        // this.SaturnDisc.drawit(viewM,projectM, globalMat);
                        
                        
                        
                        //////////////////////URANUS/////////////////////////////
                        var Uranus = glm.mat4(glm.mat3(1));
                        // ===== Scale
                        var scale = glm.vec3(0.51/2, 0.51/2, 0.51/2);
                        Uranus = glm.scale(Uranus, scale);
                        // ===== Rotation
                        var outer_rotation = counter/84.01;
                        var rotation = glm.radians(outer_rotation);
                        var axis = glm.vec3(0.0,1.0,0.0);
                        Uranus = glm.rotate(Uranus, rotation, axis);
                        // ===== Translate
                        var translation = glm.vec3(0,0,-36);
                        Uranus = glm.translate(Uranus, translation);
                        // ===== Rotation
                        Uranus = glm.rotate(Uranus, rotation, axis);
                        // ===== Final Transformation
                        this.Uranus.setModelTransformation(Uranus);      
                        // ===== Draw
                        this.Uranus.drawit(viewM,projectM, globalMat);
                        
                        
                        /////////////////////NEPTUNE////////////////////////////
                        var Neptune = glm.mat4(glm.mat3(1));
                        // ===== Scale
                        var scale = glm.vec3(0.495/2, 0.495/2, 0.495/2);
                        Neptune = glm.scale(Neptune, scale);
                        // ===== Rotation
                        var outer_rotation = counter/164.79;
                        var rotation = glm.radians(outer_rotation);
                        var axis = glm.vec3(0.0,1.0,0.0);
                        Neptune = glm.rotate(Neptune, rotation, axis);
                        // ===== Translate
                        var translation = glm.vec3(0,0,-44);
                        Neptune = glm.translate(Neptune, translation);
                        // ===== Rotation
                        Neptune = glm.rotate(Neptune, rotation, axis);
                        // ===== Final Transformation
                        this.Neptune.setModelTransformation(Neptune);      
                        // ===== Draw
                        this.Neptune.drawit(viewM,projectM, globalMat);  
                    }
                }
        </script>

        <script id="SolarSystem_Base_App">  
                class SolarSystem_Base_App extends DEECapp
                {
                    counter=0;
                    speedup = 1;
                    SunFlag = 1;
                    initialize()
                    {
                        var fragsrc = document.getElementById("fragment-shader").text;
                        var vertsrc = document.getElementById("vertex-shader").text;
                        this.shaderprog = new DEECshader(this.gl);
                        this.shaderprog.srcShaders(vertsrc,fragsrc);

                        // perform other initializations
                        this.gl.enable(this.gl.DEPTH_TEST);
                        this.gl.clearColor(0.1,0.1,0.1,1.0);
                        
                        this.solarSystem = new solarSystem(this.gl, 1);
                        this.solarSystem.setShader(this.shaderprog);

                        // ===== Set view position
                        this.viewpos = glm.vec3(0, 0, 8);
                        
                        // ===== Set lights
                        this.setLights();

                        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
                        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
                        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
                        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
                        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
                        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
                        this.projectionMa= glm.perspective(glm.radians(60),1.0,0.1,1000);      
                        
                        //////////////////////////////////////////////////////////////////////////
                        //glm.vec3(0, 0, 0): A posição da câmera no espaço 3D.                  //
                        //glm.vec3(0, 0, -1): O ponto para o qual a câmera está direcionada.    //
                        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
                        //////////////////////////////////////////////////////////////////////////
                        this.viewMa = glm.lookAt(this.viewpos, glm.vec3(2,0,0), glm.vec3(0,1,0));           

                        window.addEventListener("keypress",(evt)=>this.keyprocess(evt),false);
                    }

                    keyprocess(evt)
                    {
                        switch (evt.keyCode)
                        {
                            case 27:
                                alert("You pressed the \"Escape\" key."); 
                                break;
                            case 49: // 1 Stop
                                this.speedup = 0;     
                                break;
                            case 50: // 2 Slow
                                this.speedup = 0.2;
                                break;
                            case 51: // 3 Middle
                                this.speedup = 1;
                                break;
                            case 52: // 4 Fast
                                this.speedup = 10;
                                break;
                            case 114: // r
                            this.SunFlag = 1;
                                this.solarSystem = new solarSystem(this.gl, 1, this.SunFlag);
                                this.solarSystem.setShader(this.shaderprog, this.shaderprogTC);
                                break;
                            case 116: // t
                                this.SunFlag = 2;
                                this.solarSystem = new solarSystem(this.gl, 2, this.SunFlag);
                                this.solarSystem.setShader(this.shaderprog, this.shaderprogTC);
                                break;
                            case 121: // y
                                this.SunFlag = 3;
                                this.solarSystem = new solarSystem(this.gl, 3, this.SunFlag);
                                this.solarSystem.setShader(this.shaderprog, this.shaderprogTC);
                                break;
                            case 117: // u
                                this.SunFlag = 4;
                                this.solarSystem = new solarSystem(this.gl, 4, this.SunFlag);
                                this.solarSystem.setShader(this.shaderprog, this.shaderprogTC);
                                break;
                        }
                    }
                    
                    setLights()
                    {
                        // ===== Set parameters of light
                        this.shaderprog.startUsing();
                        // -- Ambient Light
                        this.Ambientloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Ambient");
                        var ambient_light = glm.vec3(0.2);
                        this.gl.uniform3fv(this.Ambientloc, ambient_light.array);
                        // -- Directional Light Color
                        this.LightColorloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightColor");
                        var color_light = glm.vec3(1.0, 1.0, 0.5);
                        this.gl.uniform3fv(this.LightColorloc, color_light.array);
                        // -- Directional Light Position
                        this.LightPosloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightPosition");
                        var light_pos = glm.vec3(0.0, 0.0, 0.0);
                        this.gl.uniform3fv(this.LightPosloc, light_pos.array);
                        // -- View Position
                        this.ViewPosloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ViewPos");
                        this.gl.uniform3fv(this.ViewPosloc, this.viewpos.array);
                        
                        // ----- Attenuations
                        // -- Constant Attenuation
                        this.ConstantAttenuationloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ConstantAttenuation");
                        this.gl.uniform1f(this.ConstantAttenuationloc, 0.01);
                        // -- Linear Attenuation
                        this.LinearAttenuationloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LinearAttenuation");
                        this.gl.uniform1f(this.LinearAttenuationloc, 0.0);
                        // -- Quadratic Attenuation
                        this.QuadraticAttenuationloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"QuadraticAttenuation");
                        this.gl.uniform1f(this.QuadraticAttenuationloc, 0.0);
                        // -- Setup done
                        this.shaderprog.stopUsing();
                    }
                
                    render()
                    {
                        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
                        this.counter = this.counter + this.speedup;
                        
                        this.shaderprog.startUsing();
                        
                        this.solarSystem.drawit(this.viewMa,this.projectionMa, this.counter);        
                    }
                }

                var app = new SolarSystem_Base_App('SolarSystem_Canvas');
                app.run();
        </script>
    </div>
</center>

<!-- ============================================ Bouncing Cylinder Definition ============================================ -->
<div id="BouncingCylinder">

</div>

<!-- ============================================ AR Application ============================================ -->
<center id="AR_Scene">
    <div id="AR_Scene">
        <h2> 1 - Stop | 2 - Slow | 3 - Normal | 4 - Fast (Only applies to Solar System)</h2>
        <h2> R - Spheres | T - Cubes | Y - Cylinders | U - Cones</h2>
        
        <canvas id="AR_App" width="640" height="480"
            style="border:2px solid #000000;">
            Error: Your browser does not support the HTML canvas tag.
        </canvas>

        <script id="AR_App">
            class AR_App extends DEECapp
            {
                counter = 0; counter_sun = 0; speedup = 1; SunFlag = 1;
                cubeID = 5; sphereID = 3; solarSystemID = 4; cylinderID = 2; coneID = 1; lightID = 0;
                ids = [];
                ambient_light = glm.vec3(0.6);

                initcamera()
                {
                    this.video = document.getElementById("video");
                    this.back = document.createElement('canvas');
                    this.backctx = this.back.getContext('2d');

                    if (navigator.mediaDevices === undefined) 
                    {
                        navigator.mediaDevices = {};
                    }
                
                    if (navigator.mediaDevices.getUserMedia === undefined) 
                    {
                        navigator.mediaDevices.getUserMedia = function(constraints) 
                        {
                            var getUserMedia = navigator.webkitGetUserMedia || navigator.mozGetUserMedia;
                            
                            if (!getUserMedia) 
                            {
                                return Promise.reject(new Error('getUserMedia is not implemented in this browser'));
                            }

                            return new Promise(function(resolve, reject) {getUserMedia.call(navigator, constraints, resolve, reject);});
                        }
                    }
                    
                    navigator.mediaDevices.getUserMedia({ video: true }).then(function(stream) 
                    {
                        if ("srcObject" in video)
                        {
                            video.srcObject = stream;
                        } 
                        else 
                        {
                            video.src = window.URL.createObjectURL(stream);
                        }
                    }).catch(function(err) {console.log(err.name + ": " + err.message);});
                }

                initialize()
                {
                    this.initcamera();

                    // ===== Perform initializations
                    this.gl.enable(this.gl.DEPTH_TEST);
                    this.gl.clearColor(0.1, 0.1, 0.1, 1.0);
                    
                    // ===== Initialize shader for objects
                    var fragsrc = document.getElementById("fragment-shader").text;
                    var vertsrc = document.getElementById("vertex-shader").text;
                    this.shaderprog = new DEECshader(this.gl);
                    this.shaderprog.srcShaders(vertsrc,fragsrc);
                    
                    // ===== Instantiate background video
                    var fragsrc_background = document.getElementById("fragment-shader-video").text;
                    var vertsrc_background = document.getElementById("vertex-shader-video").text;
                    this.shaderprog_video = new DEECshader(this.gl);
                    this.shaderprog_video.srcShaders(vertsrc_background,fragsrc_background);
                        
                    this.background_texture = new CGRAtexture(this.gl);

                    this.background = new square(this.gl);
                    this.background.setShader(this.shaderprog_video);
                    this.background.settexture(this.background_texture,1);

                    // ===== Instantiate cubes
                    this.myCube = new cube(this.gl);
                    this.myCube.setShader(this.shaderprog);
                    this.myCube.setMaterials(100,10);
                    this.myCube.setTexture("Diamond.jpg");

                    this.solarSystem = new solarSystem(this.gl, 1);
                    this.solarSystem.setShader(this.shaderprog);
                    this.solarSystem.setMaterials(2,50);


                    this.mySphere = new sphere(this.gl);
                    this.mySphere.setShader(this.shaderprog);
                    this.mySphere.setMaterials(10,10);
                    this.mySphere.setTexture("EarTex.jpg");

                    this.myCylinder = new cylinder(this.gl);
                    this.myCylinder.setShader(this.shaderprog);
                    this.myCylinder.setMaterials(10,10);
                    this.myCylinder.setTexture("CyTex.png");

                    this.myCone = new cone(this.gl);
                    this.myCone.setShader(this.shaderprog);
                    this.myCone.setMaterials(10,10);
                    this.myCone.setTexture("ConeTex.png");


                    // ===== Initalization of the Aruco detector
                    this.detector = new AR.Detector();
                    var modelsize = 100.0;
                    var focallength = 633.0;
                    // and of the pose estimator
                    this.posit = new POS.Posit(modelsize,focallength);

                    // ===== Set camera 
                    this.viewpos = glm.vec3(0, 0, 0); 

                    // ===== Set lights
                    this.setLights();

                    ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
                    //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
                    //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
                    //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
                    //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
                    ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
                    //this.projectionM = glm.perspective(glm.radians(45),1.33,0.1,100);
                    this.setMatrixP();              
                    // console.log(this.projectionM);

                    //////////////////////////////////////////////////////////////////////////
                    //glm.vec3(3, 3, 5): A posição da câmera no espaço 3D.                  //
                    //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
                    //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
                    //////////////////////////////////////////////////////////////////////////
                    this.viewM = glm.lookAt(this.viewpos,glm.vec3(0,0,1),glm.vec3(0,1,0));

                    window.addEventListener("keypress",(evt)=>this.keyprocess(evt),false);
                }
                
                setMatrixP()
                {
                    // this.camera_matrix =[   646.1115991593342, 0, 322.18260944421985,
                    //                         0, 647.008142040544, 242.61147838070056,
                    //                         0, 0, 1];
                    this.camera_matrix =[   632.9857064398133, 0, 327.6123818663444,
                                            0, 634.2617459480339, 238.83386640292775,
                                            0, 0, 1];

                    var u0 = this.camera_matrix[2];
                    var v0 = this.camera_matrix[5];

                    this.zfar = 300.0; this.znear = 1;
                    this.width = 640; this.height = 480;

                    // console.log(this.camera_matrix);
                    
                    // var x0 = -320; var y0 = -240;
                    var x0 = -u0; var y0 = -v0;

                    var P_0_0 = (2*this.camera_matrix[0])/this.width;
                    var P_0_1 = (-2*this.camera_matrix[1])/this.width;
                    var P_0_2 = (this.width - 2*this.camera_matrix[2] + 2*x0)/this.width;

                    var P_1_1 = (-2*this.camera_matrix[4])/this.height;
                    var P_1_2 = (this.height - 2*this.camera_matrix[5] + 2*y0)/this.height;

                    var P_2_2 = (-1*this.zfar - this.znear)/(this.zfar - this.znear);
                    var P_2_3 = (-2*this.zfar*this.znear)/(this.zfar - this.znear);

                    // this.projectionM = glm.mat4(P_0_0, P_0_1, P_0_2, 0,
                    //                             0, P_1_1, P_1_2, 0,
                    //                             0, 0, P_2_2, P_2_3,
                    //                             0, 0, -1, 0);

                    this.projectionM = glm.mat4(P_0_0   , 0     , 0     , 0,
                                                P_0_1   , P_1_1 , 0     , 0,
                                                P_0_2   , P_1_2 , P_2_2 , -1,
                                                0       , 0     , P_2_3 , 0);
                }

                setLights()
                {
                    // ===== Set parameters of light
                    this.shaderprog.startUsing();
                    // -- Ambient Light
                    this.Ambientloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Ambient");
                    // this.ambient_light = glm.vec3(0.6);
                    console.log(this.ambient_light);
                    this.gl.uniform3fv(this.Ambientloc, this.ambient_light.array);
                    // -- Directional Light Color
                    this.LightColorloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightColor");
                    this.color_light = glm.vec3(1.0);
                    this.gl.uniform3fv(this.LightColorloc, this.color_light.array);
                    // -- Directional Light Position
                    this.LightPosloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightPosition");
                    // if(this.AR_translation == undefined)
                    //     var light_pos = glm.vec3(2.0, 2.0, 2.0);
                    // else
                    //     var light_pos = glm.vec3(2.0 + this.AR_translation[0], 2.0 + this.AR_translation[1], 2.0 + this.AR_translation[2]); 
                    var light_pos = glm.vec3(-50.0, 0.0, -20.0);
                    this.gl.uniform3fv(this.LightPosloc, light_pos.array);
                    // -- View Position
                    this.ViewPosloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ViewPos");
                    this.gl.uniform3fv(this.ViewPosloc, this.viewpos.array);
                    
                    // ----- Attenuations
                    // -- Constant Attenuation
                    this.ConstantAttenuationloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ConstantAttenuation");
                    this.gl.uniform1f(this.ConstantAttenuationloc, 0.5);
                    // -- Linear Attenuation
                    this.LinearAttenuationloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LinearAttenuation");
                    this.gl.uniform1f(this.LinearAttenuationloc, 0.0);
                    // -- Quadratic Attenuation
                    this.QuadraticAttenuationloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"QuadraticAttenuation");
                    this.gl.uniform1f(this.QuadraticAttenuationloc, 0.0);
                    // -- Setup done
                    this.shaderprog.stopUsing();
                }
                
                calculateSize(srcSize, dstSize) 
                {
                    var srcRatio = srcSize.width / srcSize.height;
                    var dstRatio = dstSize.width / dstSize.height;
                    if (dstRatio > srcRatio)
                    {
                        return {width:  dstSize.height * srcRatio, height: dstSize.height};
                    } 
                    else 
                    {
                        return {width:  dstSize.width, height: dstSize.width / srcRatio};
                    }
                }

                getEulerAngles(mat) 
                {
                    let sy = Math.sqrt(mat[0] * mat[0] + mat[3] * mat[3]);

                    let singular = sy < 1e-6;

                    let x, y, z;
                    if (!singular) {
                        x = Math.atan2(mat[7], mat[8]);
                        y = Math.atan2(-mat[6], sy);
                        z = Math.atan2(mat[3], mat[0]);
                    } else {
                        x = Math.atan2(-mat[5], mat[4]);
                        y = Math.atan2(-mat[6], sy);
                        z = 0;
                    }
                    return [x * (180/Math.PI), y * (180/Math.PI), z * (180/Math.PI)]; // Returning the Euler angles in degrees
                }

                processvideo()
                {
                    if (this.video.readyState === this.video.HAVE_ENOUGH_DATA)
                    {
                        var videoSize = { width: this.video.videoWidth, height: this.video.videoHeight };
                        var canvasSize = { width: this.width, height: this.height };
                        var renderSize = this.calculateSize(videoSize, canvasSize);
                        var xOffset = (canvasSize.width - renderSize.width) / 2;
                    
                        this.back.width=this.video.videoWidth;
                        this.back.height=this.video.videoHeight;
                        this.backctx.drawImage(this.video,0,0,this.video.videoWidth,this.video.videoHeight);

                        var imagedata = this.backctx.getImageData(0,0,this.video.videoWidth,this.video.videoHeight);
                        
                        this.background_texture.update(imagedata,this.video.videoWidth,this.video.videoHeight);    

                        // Detect the markers 
                        var markers = this.detector.detect(imagedata); 

                        this.ids = [];
                        for (var i = 0; i !== markers.length; ++ i)
                        {
                            this.ids.push(markers[i].id);
                        }
                        // console.log(this.ids);
                        // console.log(markers);

                        // console.log("Markers detected = " + markers.length.toString() + "\n");
                        if (markers.length > 0)
                        {
                            for (var j = 0; j !== markers.length; ++ j)
                            {
                                var corners = markers[j].corners;

                                var pose = this.posit.pose(corners);
                                // Estamos a trabalhar em milimetros
                                var translation = pose.bestTranslation;
                                var rotation = pose.bestRotation;
                                
                                if(this.ids.includes(this.cubeID) && markers[j].id == this.cubeID)
                                {
                                    this.cube_translation = translation;
                                    // this.cube_rotation = rotation;
                                    this.cube_rotation = this.getEulerAngles(rotation.flat());
                                }

                                if(this.ids.includes(this.solarSystemID) && markers[j].id == this.solarSystemID)
                                {
                                    this.solarSystem_translation = translation;
                                    this.solarSystem_rotation = this.getEulerAngles(rotation.flat());
                                }

                                if(this.ids.includes(this.sphereID) && markers[j].id == this.sphereID)   
                                {
                                    this.sphere_translation = translation;
                                    this.sphere_rotation = this.getEulerAngles(rotation.flat());
                                }
                                
                                if(this.ids.includes(this.cylinderID) && markers[j].id == this.cylinderID)
                                {
                                    this.cylinder_translation = translation;
                                    this.cylinder_rotation = this.getEulerAngles(rotation.flat());
                                }

                                if(this.ids.includes(this.coneID) && markers[j].id == this.coneID)
                                {
                                    this.cone_translation = translation;
                                    this.cone_rotation = this.getEulerAngles(rotation.flat());
                                }
                            
                                if(this.ids.includes(this.lightID) && markers[j].id == this.lightID)
                                {
                                    this.ambient_light = glm.vec3(0.6);
                                }

                                // console.log(this.ambient_light.toString());
                            }
                        }
                    }      
                }

                keyprocess(evt)
                    {
                        switch (evt.keyCode)
                        {
                            case 27:
                                alert("You pressed the \"Escape\" key."); 
                                break;
                            case 49: // 1 Stop
                                this.speedup = 0;     
                                break;
                            case 50: // 2 Slow
                                this.speedup = 0.2;
                                break;
                            case 51: // 3 Middle
                                this.speedup = 1;
                                break;
                            case 52: // 4 Fast
                                this.speedup = 10;
                                break;
                            case 114: // r
                            this.SunFlag = 1;
                                this.solarSystem = new solarSystem(this.gl, 1, this.SunFlag);
                                this.solarSystem.setShader(this.shaderprog, this.shaderprogTC);
                                break;
                            case 116: // t
                                this.SunFlag = 2;
                                this.solarSystem = new solarSystem(this.gl, 2, this.SunFlag);
                                this.solarSystem.setShader(this.shaderprog, this.shaderprogTC);
                                break;
                            case 121: // y
                                this.SunFlag = 3;
                                this.solarSystem = new solarSystem(this.gl, 3, this.SunFlag);
                                this.solarSystem.setShader(this.shaderprog, this.shaderprogTC);
                                break;
                            case 117: // u
                                this.SunFlag = 4;
                                this.solarSystem = new solarSystem(this.gl, 4, this.SunFlag);
                                this.solarSystem.setShader(this.shaderprog, this.shaderprogTC);
                                break;
                        }
                    }
                           
                render()
                {
                    this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
                    this.ambient_light = glm.vec3(1.0);
                    this.processvideo();
                    this.setLights();
                    this.counter++;
                    
                    this.counter_sun = this.counter_sun + this.speedup;

                    // ==================== Background ====================
                    this.gl.disable(this.gl.DEPTH_TEST);
                    this.background.drawit();
                    this.gl.enable(this.gl.DEPTH_TEST);


                    // ==================== Cube ====================
                    // ===== Model
                    var model = glm.mat4(glm.mat3(1.0));
                    // ===== Translations
                    if (this.cube_translation != undefined)
                        var AR_translation = glm.vec3(this.cube_translation[0]/10, this.cube_translation[1]/10, this.cube_translation[2]/10);
                    else
                        var AR_translation = glm.vec3(0.0, 0.0, 100.0);
                    model = glm.translate(model, AR_translation);
                    // ===== Rotation
                    var referential = [glm.vec3(1.0, 0.0, 0.0), glm.vec3(0.0, 1.0, 0.0), glm.vec3(0.0, 0.0, 1.0)];
                    for(var i=0; i<3; i++)
                    {
                        if (this.cube_rotation != undefined)
                            this.AR_rotation = glm.radians(this.cube_rotation[i]);
                        else
                            this.AR_rotation = glm.radians(0.0);
                        var axis = referential[i];
                        model = glm.rotate(model, this.AR_rotation, axis);
                    }
                    // ===== Scale
                    var scale = glm.vec3(5.0);
                    model = glm.scale(model, scale);
                    // ===== Final Transformation
                    this.myCube.setModelTransformation(model); 
                    // ===== Draw Cube
                    if(this.ids.includes(this.cubeID))
                        this.myCube.drawit(this.viewM,this.projectionM);


                    // ==================== Solar System ====================
                    // ===== Model
                    var model = glm.mat4(glm.mat3(1.0));
                    // ===== Translations
                    if (this.solarSystem_translation != undefined)
                        var AR_translation = glm.vec3(this.solarSystem_translation[0]/10, this.solarSystem_translation[1]/10, this.solarSystem_translation[2]/10);
                    else
                        var AR_translation = glm.vec3(0.0, 0.0, 100.0);
                    model = glm.translate(model, AR_translation);
                    // ===== Rotation
                    var referential = [glm.vec3(1.0, 0.0, 0.0), glm.vec3(0.0, 1.0, 0.0), glm.vec3(0.0, 0.0, 1.0)];
                    for(var i=0; i<3; i++)
                    {
                        if (this.solarSystem_rotation != undefined)
                            this.AR_rotation = glm.radians(this.solarSystem_rotation[i]);
                        else
                            this.AR_rotation = glm.radians(0.0);
                        var axis = referential[i];
                        model = glm.rotate(model, this.AR_rotation, axis);
                    }
                    // ===== Scale
                    var scale = glm.vec3(3.0);
                    model = glm.scale(model, scale);
                    // ===== Final Transformation
                    this.solarSystem.setModelTransformation(model); 
                    // ===== Draw Cube
                    if(this.ids.includes(this.solarSystemID))
                        this.solarSystem.drawit(this.viewM,this.projectionM, this.counter_sun);


                    // ==================== Sphere ====================
                    // ===== Model
                    var model = glm.mat4(glm.mat3(1.0));
                    // ===== Translations
                    if (this.sphere_translation != undefined)
                        this.AR_translation = glm.vec3(this.sphere_translation[0]/10, this.sphere_translation[1]/10, this.sphere_translation[2]/10);
                    else
                        this.AR_translation = glm.vec3(220.0, 0.0, 0.0);
                    model = glm.translate(model, this.AR_translation);
                    // ===== Rotation
                    var referential = [glm.vec3(1.0, 0.0, 0.0), glm.vec3(0.0, 1.0, 0.0), glm.vec3(0.0, 0.0, 1.0)];
                    for(var i=0; i<3; i++)
                    {
                        if (this.sphere_rotation != undefined)
                            this.AR_rotation = glm.radians(this.sphere_rotation[i]);
                        else
                            this.AR_rotation = glm.radians(0.0);
                        var axis = referential[i];
                        model = glm.rotate(model, this.AR_rotation, axis);
                    }
                    // ===== Scale
                    var scale = glm.vec3(10.0);
                    model = glm.scale(model, scale);
                    // ===== Final Transformation
                    this.mySphere.setModelTransformation(model); 
                    // ===== Draw Sphere
                    if(this.ids.includes(this.sphereID))
                        this.mySphere.drawit(this.viewM,this.projectionM);

                    
                    // ==================== Cylinder ====================
                    // ===== Model
                    var model = glm.mat4(glm.mat3(1.0));
                    // ===== Translations
                    if (this.cylinder_translation != undefined)
                        this.AR_translation = glm.vec3(this.cylinder_translation[0]/10, this.cylinder_translation[1]/10, this.cylinder_translation[2]/10);
                    else
                        this.AR_translation = glm.vec3(0.0, 0.0, 100.0);
                    model = glm.translate(model, this.AR_translation);
                    // ===== Rotation
                    var referential = [glm.vec3(1.0, 0.0, 0.0), glm.vec3(0.0, 1.0, 0.0), glm.vec3(0.0, 0.0, 1.0)];
                    for(var i=0; i<3; i++)
                    {
                        if (this.cylinder_rotation != undefined)
                            this.AR_rotation = glm.radians(this.cylinder_rotation[i]);
                        else
                            this.AR_rotation = glm.radians(0.0);
                        var axis = referential[i];
                        model = glm.rotate(model, this.AR_rotation, axis);
                    }
                    // ===== Scale
                    var scale = glm.vec3(5.0);
                    model = glm.scale(model, scale);
                    // ===== Final Transformation
                    this.myCylinder.setModelTransformation(model);
                    // ===== Draw Cylinder
                    if(this.ids.includes(this.cylinderID))
                        this.myCylinder.drawit(this.viewM,this.projectionM);


                    // ==================== Cone ====================
                    // ===== Model
                    var model = glm.mat4(glm.mat3(1.0));
                    // ===== Translations
                    if (this.cone_translation != undefined)
                        this.AR_translation = glm.vec3(this.cone_translation[0]/10, this.cone_translation[1]/10, this.cone_translation[2]/10);
                    else
                        this.AR_translation = glm.vec3(0.0, 0.0, 100.0);
                    model = glm.translate(model, this.AR_translation);
                    // ===== Rotation
                    var referential = [glm.vec3(1.0, 0.0, 0.0), glm.vec3(0.0, 1.0, 0.0), glm.vec3(0.0, 0.0, 1.0)];
                    for(var i=0; i<3; i++)
                    {
                        if (this.cone_rotation != undefined)
                            this.AR_rotation = glm.radians(this.cone_rotation[i]);
                        else
                            this.AR_rotation = glm.radians(0.0);
                        var axis = referential[i];
                        model = glm.rotate(model, this.AR_rotation, axis);
                    }
                    // ===== Scale
                    var scale = glm.vec3(5.0);
                    model = glm.scale(model, scale);
                    // ===== Final Transformation
                    this.myCone.setModelTransformation(model);
                    // ===== Draw Cone
                    if(this.ids.includes(this.coneID))
                        this.myCone.drawit(this.viewM,this.projectionM);

                    
                    // Get intermediete matrix
                    // var MVP_draft = this.projectionM['*'](this.viewM)['*'](glm.vec4(this.AR_translation[0], this.AR_translation[1], this.AR_translation[2], 1.0));
                    // MVP_draft['/='](MVP_draft[3]);
                    // console.log("MVP_DRAFT = " + MVP_draft.toString());                
                }
            }

            var app = new AR_App('AR_App');

            app.run();

        </script>
    </div>
</center>
